In [1]:
import requests
import pandas as pd
import time
import random
from bs4 import BeautifulSoup
import datetime
import re
import os

In [3]:
import requests
import pandas as pd
import time
import random
from bs4 import BeautifulSoup
import datetime
import re
import os

class BossSpiderAPI:
    def __init__(self):
        """金融行业职位爬虫 - 支持断点续爬 (优化版)"""
        self.base_url = "https://www.zhipin.com/wapi/zpgeek/search/joblist.json"
        
        # 优化 1: 使用 Session 对象，维持连接池
        self.session = requests.Session()
        
        # 优化 2: 准备 User-Agent 池
        self.ua_list = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 Edg/128.0.0.0"
        ]

        self.headers = {
            "User-Agent": random.choice(self.ua_list),
            "Referer": "https://www.zhipin.com/web/geek/job",
            "Accept": "application/json, text/plain, */*",
            "Content-Type": "application/x-www-form-urlencoded",
            "Origin": "https://www.zhipin.com", # 补充 Origin
            "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8", # 补充语言头
            # 注意：Cookie 需要定期更新
            "Cookie": "ab_guid=78d319c7-13e6-4a47-baf0-f120a7373d6f; lastCity=100010000; wt2=Dkk1F98VQmqMn0UgGwTzC7o7OoHUbYqVCcZHsIQC0DajHRfhzMgYzgV38ljcbzyds0dOdNNbVyWhaO38G_gxErg~~; wbg=0; zp_at=LZazoHzQtE86zsLEFo3PmN_T2jRcXSEvG-dxQ9Lh-JI~; bst=V2R9MnF-L5219pVtRuyRgeLCm07DrVwig~|R9MnF-L5219pVtRuyRgeLCm07DrXwyQ~; __zp_stoken__=82aegw5ZfxIIcSRISbmsXEHhRw49VZFpTUMK%2BaGhaXFDCqcODw4bCjcKiwodVw4%2FDj8ONwpxVwrtbwrHCtsKkVsK6w4LCscODwq9SxINbwqrDiMKlxKvDvW3CsMKAWsOMwp9AOsONxIzFh8WGxYXFjMOtxIfFhsWFxYzDjcWHxYbFhcOgxI3Dn8OWxYXFjMWOxYfFhsWFxYxJTkBBS0dOP8K1xYxINkdETUzDjMKAw4Bmw4bCgcODb8OHwovDgmxJKiwWF2EaFhFjY2hqYm0eGGsdHWEdGhJsHhoTbWIdGmgSETQ%2Bw4vDhhNZGT4Sw4fDqBnDi0gTw5RiGy9Lw4LDkCYxQsSAVsKxQE%2FDhMWMTEBHQ01MTUAmR0tBTkNKREFANkvDh2vDl2saJUrDgkkvQ05HRE1MMmZDSkJDQENKQEFON0pEwo82TsOJwpfDhsOrQ0o%3D"
        }
        
        # 更新 Session 的 headers
        self.session.headers.update(self.headers)
        
        self.detail_headers = {
            "User-Agent": self.headers["User-Agent"],
            "Cookie": self.headers["Cookie"],
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
            "Upgrade-Insecure-Requests": "1"
        }
        
        self.finance_pattern = re.compile(
            r"(金融|银行|券商|证券|基金|保险|期货|信托|投行|资管|理财|财富|"
            r"小贷|消金|保理|租赁|征信|清算|支付|互金|金融科技|FinTech)",
            re.IGNORECASE
        )
        
        self.progress_file = "job_data_IN_PROGRESS.xlsx"
        self.state_file = "crawl_state.txt"
        
        self.data_list = []
        self.seen_ids = set()
        
        self.current_keyword_index = 0
        self.current_page = 1

    def is_finance_related(self, job):
        """判断是否为金融相关职位"""
        text = " ".join([
            job.get("brandIndustry", "") or "",
            job.get("brandName", "") or "",
            job.get("jobName", "") or ""
        ])
        return bool(self.finance_pattern.search(text))

    def load_crawl_state(self, keywords):
        """加载爬取状态（当前关键词和页码）"""
        if not os.path.exists(self.state_file):
            return 0, 1
        
        try:
            with open(self.state_file, 'r', encoding='utf-8') as f:
                lines = f.readlines()
                keyword = lines[0].strip()
                page = int(lines[1].strip())
                
                if keyword in keywords:
                    idx = keywords.index(keyword)
                    print(f"📍 恢复爬取进度：关键词 [{keyword}] 第 {page} 页")
                    return idx, page
                else:
                    print(f"⚠️ 状态文件中的关键词 [{keyword}] 不在当前列表中，从头开始")
                    return 0, 1
        except Exception as e:
            print(f"⚠️ 读取状态文件失败: {e}，从头开始")
            return 0, 1

    def save_crawl_state(self, keyword, page):
        """保存爬取状态"""
        try:
            with open(self.state_file, 'w', encoding='utf-8') as f:
                f.write(f"{keyword}\n{page}")
        except Exception as e:
            print(f"⚠️ 保存状态失败: {e}")

    def fetch_data(self, query="金融", start_page=1, max_pages=5):
        """抓取职位列表数据"""
        for page in range(start_page, max_pages + 1):
            print(f"\n🔍 抓取关键词 [{query}] 列表第 {page} 页...")
            
            params = {
                "scene": "1",
                "query": query,
                "city": "100010000",
                "page": page,
                "pageSize": 30
            }
            
            try:
                # 优化 4: 使用 session 发送请求
                resp = self.session.post(
                    self.base_url, 
                    data=params,
                    timeout=15
                )
                resp.raise_for_status()
            except requests.RequestException as e:
                print(f"❌ 请求失败：{e}")
                continue

            try:
                result = resp.json()
                code = result.get("code")
                msg = result.get("message") or result.get("msg")
                
                if code == 37:
                    print(f"⚠️ 触发风控 (code: 37)！")
                    print(f"📍 当前进度：关键词 [{query}] 第 {page} 页")
                    self.save_crawl_state(query, page)
                    self.save_progress()
                    return False
                elif code not in (0, "0", None):
                    print(f"⚠️ 接口异常 code: {code}, message: {msg}")
                    break
                    
            except Exception as e:
                print(f"❌ 解析响应失败: {e}")
                continue

            job_list = result.get("zpData", {}).get("jobList", [])
            
            if not job_list:
                print("⚠️ 本页无数据，进入下一个关键词")
                break
            
            print(f"✅ 获取到 {len(job_list)} 个职位")
            
            finance_count = 0
            for job in job_list:
                job_id = job.get("encryptJobId")
                
                if not job_id or job_id in self.seen_ids:
                    continue
                
                if not self.is_finance_related(job):
                    continue
                
                finance_count += 1
                
                item = {
                    "职位": job.get("jobName"),
                    "公司": job.get("brandName"),
                    "薪资": job.get("salaryDesc"),
                    "地区": job.get("cityName"),
                    "经验": job.get("jobExperience"),
                    "学历": job.get("jobDegree"),
                    "公司规模": job.get("brandScaleName"),
                    "行业": job.get("brandIndustry"),
                    "福利标签": ",".join(job.get("welfareList", []) or []),
                    "技能标签": ",".join(job.get("skills", []) or []),
                    "职位描述": "",
                    "job_id": job_id
                }
                
                self.data_list.append(item)
                self.seen_ids.add(job_id)
            
            print(f"📊 本页符合条件: {finance_count} 个 | 累计: {len(self.data_list)} 个")
            
            self.save_crawl_state(query, page + 1)
            self.save_progress()
            
            time.sleep(random.uniform(2.0, 4.0))
        
        return True

    def get_job_detail(self, job_id):
        """获取职位详情描述"""
        url = f"https://www.zhipin.com/job_detail/{job_id}.html"
        try:
            # 优化 5: 使用 session 获取详情
            resp = self.session.get(url, headers=self.detail_headers, timeout=10)
            if resp.status_code == 404:
                return "职位已关闭或不存在"
            if resp.status_code != 200:
                print(f"  ⚠️ 详情页状态码: {resp.status_code}")
                return None
            soup = BeautifulSoup(resp.text, 'html.parser')
            desc_tag = soup.select_one('.job-sec-text')
            return desc_tag.text.strip() if desc_tag else ""
        except requests.exceptions.RequestException as e:
            print(f"  ⚠️ 详情页请求失败: {e}")
            return None

    def load_progress(self):
        """从进度文件中加载数据"""
        print(f"\n🔄 正在从 {self.progress_file} 加载进度...")
        try:
            df = pd.read_excel(self.progress_file)
            
            df['职位描述'] = df['职位描述'].fillna('')
            df['job_id'] = df['job_id'].fillna('')
            
            self.data_list = df.to_dict('records')
            
            for item in self.data_list:
                if item.get('job_id'):
                    self.seen_ids.add(item['job_id'])
            
            completed = sum(1 for item in self.data_list 
                           if item.get('职位描述') and 
                           str(item['职位描述']).strip() and 
                           str(item['职位描述']).strip().lower() != 'nan' and
                           len(str(item['职位描述']).strip()) > 10)
            
            print(f"✅ 加载成功，已恢复 {len(self.data_list)} 条记录。")
            print(f"   其中 {completed} 条已有描述，{len(self.data_list) - completed} 条待抓取。")
            
        except FileNotFoundError:
            print("ℹ️ 未找到进度文件，将创建新文件。")
        except Exception as e:
            print(f"❌ 加载进度文件失败: {e}")

    def save_progress(self):
        """将当前数据保存到进度文件"""
        if not self.data_list:
            return
        try:
            df = pd.DataFrame(self.data_list)
            df.to_excel(self.progress_file, index=False, na_rep='')
        except Exception as e:
            print(f"  ❌ 保存进度失败: {e}")

    def fetch_all_details_resumable(self, start_index=0):
        """可恢复地抓取所有详情"""
        print(f"\n{'='*60}")
        print(f"🚀 (阶段2) 开始抓取职位详细描述...")
        if start_index > 0:
            print(f"📍 从第 {start_index + 1} 条开始（跳过前 {start_index} 条）")
        print(f"{'='*60}")
        
        fetched_count = 0
        skipped_count = 0
        failed_count = 0
        
        # 优化 6: 批量保存计数器
        unsaved_changes = 0 
        BATCH_SIZE = 10  # 每抓取 10 条保存一次
        
        for i, item in enumerate(self.data_list):
            if i < start_index:
                skipped_count += 1
                if skipped_count % 100 == 0:
                    print(f"  [快速跳过] 已跳过 {skipped_count} 条...")
                continue
            
            desc = item.get("职位描述", "")
            desc_str = str(desc).strip() if desc else ""
            
            if desc_str and desc_str.lower() != 'nan' and len(desc_str) > 10:
                skipped_count += 1
                if skipped_count % 20 == 1:
                    print(f"  ({i+1}/{len(self.data_list)}) [跳过] 已有描述")
                continue
            
            print(f"  ({i+1}/{len(self.data_list)}) [抓取] {item['职位']} - {item['公司']}")
            
            job_id = item.get("job_id")
            if not job_id:
                continue
            
            new_desc = self.get_job_detail(job_id)
            
            if new_desc is None:
                failed_count += 1
                print("  ❌ 详情页抓取失败，可能已触发风控！")
                
                if failed_count >= 3:
                    print(f"\n  🛑 连续失败 {failed_count} 次，停止抓取。")
                    self.save_progress()
                    return i
                else:
                    print(f"  ⚠️ 将继续尝试下一条（失败计数：{failed_count}/3）")
                    time.sleep(random.uniform(5.0, 8.0))
                    continue
            
            failed_count = 0
            item["职位描述"] = new_desc
            fetched_count += 1
            unsaved_changes += 1
            
            # 优化 7: 批量保存逻辑
            if unsaved_changes >= BATCH_SIZE:
                print(f"  💾 批量保存进度 (已新增 {unsaved_changes} 条)...")
                self.save_progress()
                unsaved_changes = 0
            
            time.sleep(random.uniform(3.0, 6.0))
        
        if unsaved_changes > 0:
            self.save_progress()
            
        print("\n✅ (阶段2) 详情抓取完毕。")
        print(f"   跳过: {skipped_count} 条 | 新增: {fetched_count} 条 | 失败: {failed_count} 条")
        return -1

    def save_final_excel(self):
        """保存为最终的 Excel 文件"""
        if not self.data_list:
            print("\n❌ 没有数据可保存")
            return
        
        final_data = [item.copy() for item in self.data_list]
        for item in final_data:
            if 'job_id' in item:
                del item['job_id']
                
        df = pd.DataFrame(final_data)
        
        filename = f"金融行业岗位_完整版_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df.to_excel(filename, index=False)
        
        print(f"\n{'='*60}")
        print(f"🎉 全部完成！已保存最终文件：{filename}")
        print(f"    📦 共 {len(df)} 条职位")
        print(f"    📋 包含列: {', '.join(df.columns.tolist())}")

    def run(self, keywords=["金融"], max_pages_per_kw=10, start_detail_index=0):
        """运行爬虫"""
        if os.path.exists(self.progress_file):
            self.load_progress()
        
        start_kw_idx, start_page = self.load_crawl_state(keywords)
        
        print(f"\n🚀 (阶段1) 开始/继续抓取职位列表...")
        
        for kw_idx in range(start_kw_idx, len(keywords)):
            keyword = keywords[kw_idx]
            page = start_page if kw_idx == start_kw_idx else 1
            
            print(f"\n{'='*40}")
            print(f"关键词 ({kw_idx+1}/{len(keywords)}): {keyword}")
            print(f"{'='*40}")
            
            success = self.fetch_data(query=keyword, start_page=page, max_pages=max_pages_per_kw)
            
            if not success:
                print("\n⚠️ 触发风控，已保存当前进度。")
                return
            
            if kw_idx < len(keywords) - 1:
                time.sleep(random.uniform(3.0, 5.0))
        
        print("\n👍 (阶段1) 职位列表抓取完毕。")
        
        if self.data_list:
            self.fetch_all_details_resumable(start_index=start_detail_index)
        
        self.save_final_excel()
        
        if os.path.exists(self.state_file):
            os.remove(self.state_file)
            print(f"   🗑️  已删除状态文件 {self.state_file}")

if __name__ == "__main__":
    spider = BossSpiderAPI()
    
    keywords = ["银行","保险"]
    
    # 每个关键词抓取 10# filepath: e:\Programming\ds and ai\project\Data-Science-and-AI\boss\boss.ipynb
import requests
import pandas as pd
import time
import random
from bs4 import BeautifulSoup
import datetime
import re
import os

class BossSpiderAPI:
    def __init__(self):
        """金融行业职位爬虫 - 支持断点续爬 (优化版)"""
        self.base_url = "https://www.zhipin.com/wapi/zpgeek/search/joblist.json"
        
        # 优化 1: 使用 Session 对象，维持连接池
        self.session = requests.Session()
        
        # 优化 2: 准备 User-Agent 池
        self.ua_list = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 Edg/128.0.0.0"
        ]

        self.headers = {
            "User-Agent": random.choice(self.ua_list),
            "Referer": "https://www.zhipin.com/web/geek/job",
            "Accept": "application/json, text/plain, */*",
            "Content-Type": "application/x-www-form-urlencoded",
            "Origin": "https://www.zhipin.com", # 补充 Origin
            "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8", # 补充语言头
            # 注意：Cookie 需要定期更新
            "Cookie": "ab_guid=78d319c7-13e6-4a47-baf0-f120a7373d6f; lastCity=100010000; wt2=Dkk1F98VQmqMn0UgGwTzC7o7OoHUbYqVCcZHsIQC0DajHRfhzMgYzgV38ljcbzyds0dOdNNbVyWhaO38G_gxErg~~; wbg=0; zp_at=LZazoHzQtE86zsLEFo3PmN_T2jRcXSEvG-dxQ9Lh-JI~; bst=V2R9MnF-L5219pVtRuyRgYLSy27DrRzSk~|R9MnF-L5219pVtRuyRgYLSy27Drexyw~; __zp_stoken__=0a91gPEDDpMK5wpbCvUMvE0s8NEc8RzxAPT5BPEBfLUM5RThBQh9Xwr3Cig0kWMOIScK5Qzs9PEs8QEc8OkcrPUBDOUBDQ0PEusOFQEPCsFDEgUMsJcKEw4DCjw8iZMOVDMO6w4MNw6PCuA0pDWvCvQ1Gwrw7KBMPXRAXXmYUEBJgDxMVYxcWXBISZ19jYl1dChcQYwkTGSBLw67Cv8Kuwr3Ds8K9wqjDgcOywrzCrsOLOUU4OTM9xYbCrzFLQEdDPEs%2BxYbEv8S6xLzEv8WGxL%2FDisOUw7rDmsS%2FxLrEvMK6xYbEv8S6w7zDmsWGxL%2FEusS8w7nDhTY8wpvCucKiw6fCsV7DqsSnwp%2FCrsKTwqnCt1DCpcK6wp7CuMO8dMKzdsK3XMKtwqpMWsKZUGLCgMKYwofCusK4wp5KUFZhXcK0TE1XYU3DgVNxFRMRelgTQRBkPsOT"
        }
        
        # 更新 Session 的 headers
        self.session.headers.update(self.headers)
        
        self.detail_headers = {
            "User-Agent": self.headers["User-Agent"],
            "Cookie": self.headers["Cookie"],
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
            "Upgrade-Insecure-Requests": "1"
        }
        
        self.finance_pattern = re.compile(
            r"(金融|银行|券商|证券|基金|保险|期货|信托|投行|资管|理财|财富|"
            r"小贷|消金|保理|租赁|征信|清算|支付|互金|金融科技|FinTech)",
            re.IGNORECASE
        )
        
        self.progress_file = "job_data_IN_PROGRESS.xlsx"
        self.state_file = "crawl_state.txt"
        
        self.data_list = []
        self.seen_ids = set()
        
        self.current_keyword_index = 0
        self.current_page = 1

    def is_finance_related(self, job):
        """判断是否为金融相关职位"""
        text = " ".join([
            job.get("brandIndustry", "") or "",
            job.get("brandName", "") or "",
            job.get("jobName", "") or ""
        ])
        return bool(self.finance_pattern.search(text))

    def load_crawl_state(self, keywords):
        """加载爬取状态（当前关键词和页码）"""
        if not os.path.exists(self.state_file):
            return 0, 1
        
        try:
            with open(self.state_file, 'r', encoding='utf-8') as f:
                lines = f.readlines()
                keyword = lines[0].strip()
                page = int(lines[1].strip())
                
                if keyword in keywords:
                    idx = keywords.index(keyword)
                    print(f"📍 恢复爬取进度：关键词 [{keyword}] 第 {page} 页")
                    return idx, page
                else:
                    print(f"⚠️ 状态文件中的关键词 [{keyword}] 不在当前列表中，从头开始")
                    return 0, 1
        except Exception as e:
            print(f"⚠️ 读取状态文件失败: {e}，从头开始")
            return 0, 1

    def save_crawl_state(self, keyword, page):
        """保存爬取状态"""
        try:
            with open(self.state_file, 'w', encoding='utf-8') as f:
                f.write(f"{keyword}\n{page}")
        except Exception as e:
            print(f"⚠️ 保存状态失败: {e}")

    def fetch_data(self, query="金融", start_page=1, max_pages=5):
        """抓取职位列表数据"""
        for page in range(start_page, max_pages + 1):
            print(f"\n🔍 抓取关键词 [{query}] 列表第 {page} 页...")
            
            params = {
                "scene": "1",
                "query": query,
                "city": "100010000",
                "page": page,
                "pageSize": 30
            }
            
            try:
                # 优化 4: 使用 session 发送请求
                resp = self.session.post(
                    self.base_url, 
                    data=params,
                    timeout=15
                )
                resp.raise_for_status()
            except requests.RequestException as e:
                print(f"❌ 请求失败：{e}")
                continue

            try:
                result = resp.json()
                code = result.get("code")
                msg = result.get("message") or result.get("msg")
                
                if code == 37:
                    print(f"⚠️ 触发风控 (code: 37)！")
                    print(f"📍 当前进度：关键词 [{query}] 第 {page} 页")
                    self.save_crawl_state(query, page)
                    self.save_progress()
                    return False
                elif code not in (0, "0", None):
                    print(f"⚠️ 接口异常 code: {code}, message: {msg}")
                    break
                    
            except Exception as e:
                print(f"❌ 解析响应失败: {e}")
                continue

            job_list = result.get("zpData", {}).get("jobList", [])
            
            if not job_list:
                print("⚠️ 本页无数据，进入下一个关键词")
                break
            
            print(f"✅ 获取到 {len(job_list)} 个职位")
            
            finance_count = 0
            for job in job_list:
                job_id = job.get("encryptJobId")
                
                if not job_id or job_id in self.seen_ids:
                    continue
                
                if not self.is_finance_related(job):
                    continue
                
                finance_count += 1
                
                item = {
                    "职位": job.get("jobName"),
                    "公司": job.get("brandName"),
                    "薪资": job.get("salaryDesc"),
                    "地区": job.get("cityName"),
                    "经验": job.get("jobExperience"),
                    "学历": job.get("jobDegree"),
                    "公司规模": job.get("brandScaleName"),
                    "行业": job.get("brandIndustry"),
                    "福利标签": ",".join(job.get("welfareList", []) or []),
                    "技能标签": ",".join(job.get("skills", []) or []),
                    "职位描述": "",
                    "job_id": job_id
                }
                
                self.data_list.append(item)
                self.seen_ids.add(job_id)
            
            print(f"📊 本页符合条件: {finance_count} 个 | 累计: {len(self.data_list)} 个")
            
            self.save_crawl_state(query, page + 1)
            self.save_progress()
            
            time.sleep(random.uniform(2.0, 4.0))
        
        return True

    def get_job_detail(self, job_id):
        """获取职位详情描述"""
        url = f"https://www.zhipin.com/job_detail/{job_id}.html"
        try:
            # 优化 5: 使用 session 获取详情
            resp = self.session.get(url, headers=self.detail_headers, timeout=10)
            if resp.status_code == 404:
                return "职位已关闭或不存在"
            if resp.status_code != 200:
                print(f"  ⚠️ 详情页状态码: {resp.status_code}")
                return None
            soup = BeautifulSoup(resp.text, 'html.parser')
            desc_tag = soup.select_one('.job-sec-text')
            return desc_tag.text.strip() if desc_tag else ""
        except requests.exceptions.RequestException as e:
            print(f"  ⚠️ 详情页请求失败: {e}")
            return None

    def load_progress(self):
        """从进度文件中加载数据"""
        print(f"\n🔄 正在从 {self.progress_file} 加载进度...")
        try:
            df = pd.read_excel(self.progress_file)
            
            df['职位描述'] = df['职位描述'].fillna('')
            df['job_id'] = df['job_id'].fillna('')
            
            self.data_list = df.to_dict('records')
            
            for item in self.data_list:
                if item.get('job_id'):
                    self.seen_ids.add(item['job_id'])
            
            completed = sum(1 for item in self.data_list 
                           if item.get('职位描述') and 
                           str(item['职位描述']).strip() and 
                           str(item['职位描述']).strip().lower() != 'nan' and
                           len(str(item['职位描述']).strip()) > 10)
            
            print(f"✅ 加载成功，已恢复 {len(self.data_list)} 条记录。")
            print(f"   其中 {completed} 条已有描述，{len(self.data_list) - completed} 条待抓取。")
            
        except FileNotFoundError:
            print("ℹ️ 未找到进度文件，将创建新文件。")
        except Exception as e:
            print(f"❌ 加载进度文件失败: {e}")

    def save_progress(self):
        """将当前数据保存到进度文件"""
        if not self.data_list:
            return
        try:
            df = pd.DataFrame(self.data_list)
            df.to_excel(self.progress_file, index=False, na_rep='')
        except Exception as e:
            print(f"  ❌ 保存进度失败: {e}")

    def fetch_all_details_resumable(self, start_index=0):
        """可恢复地抓取所有详情"""
        print(f"\n{'='*60}")
        print(f"🚀 (阶段2) 开始抓取职位详细描述...")
        if start_index > 0:
            print(f"📍 从第 {start_index + 1} 条开始（跳过前 {start_index} 条）")
        print(f"{'='*60}")
        
        fetched_count = 0
        skipped_count = 0
        failed_count = 0
        
        # 优化 6: 批量保存计数器
        unsaved_changes = 0 
        BATCH_SIZE = 10  # 每抓取 10 条保存一次
        
        for i, item in enumerate(self.data_list):
            if i < start_index:
                skipped_count += 1
                if skipped_count % 100 == 0:
                    print(f"  [快速跳过] 已跳过 {skipped_count} 条...")
                continue
            
            desc = item.get("职位描述", "")
            desc_str = str(desc).strip() if desc else ""
            
            if desc_str and desc_str.lower() != 'nan' and len(desc_str) > 10:
                skipped_count += 1
                if skipped_count % 20 == 1:
                    print(f"  ({i+1}/{len(self.data_list)}) [跳过] 已有描述")
                continue
            
            print(f"  ({i+1}/{len(self.data_list)}) [抓取] {item['职位']} - {item['公司']}")
            
            job_id = item.get("job_id")
            if not job_id:
                continue
            
            new_desc = self.get_job_detail(job_id)
            
            if new_desc is None:
                failed_count += 1
                print("  ❌ 详情页抓取失败，可能已触发风控！")
                
                if failed_count >= 3:
                    print(f"\n  🛑 连续失败 {failed_count} 次，停止抓取。")
                    self.save_progress()
                    return i
                else:
                    print(f"  ⚠️ 将继续尝试下一条（失败计数：{failed_count}/3）")
                    time.sleep(random.uniform(5.0, 8.0))
                    continue
            
            failed_count = 0
            item["职位描述"] = new_desc
            fetched_count += 1
            unsaved_changes += 1
            
            # 优化 7: 批量保存逻辑
            if unsaved_changes >= BATCH_SIZE:
                print(f"  💾 批量保存进度 (已新增 {unsaved_changes} 条)...")
                self.save_progress()
                unsaved_changes = 0
            
            time.sleep(random.uniform(3.0, 6.0))
        
        if unsaved_changes > 0:
            self.save_progress()
            
        print("\n✅ (阶段2) 详情抓取完毕。")
        print(f"   跳过: {skipped_count} 条 | 新增: {fetched_count} 条 | 失败: {failed_count} 条")
        return -1

    def save_final_excel(self):
        """保存为最终的 Excel 文件"""
        if not self.data_list:
            print("\n❌ 没有数据可保存")
            return
        
        final_data = [item.copy() for item in self.data_list]
        for item in final_data:
            if 'job_id' in item:
                del item['job_id']
                
        df = pd.DataFrame(final_data)
        
        filename = f"金融行业岗位_完整版_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df.to_excel(filename, index=False)
        
        print(f"\n{'='*60}")
        print(f"🎉 全部完成！已保存最终文件：{filename}")
        print(f"    📦 共 {len(df)} 条职位")
        print(f"    📋 包含列: {', '.join(df.columns.tolist())}")

    def run(self, keywords=["金融"], max_pages_per_kw=10, start_detail_index=0):
        """运行爬虫"""
        if os.path.exists(self.progress_file):
            self.load_progress()
        
        start_kw_idx, start_page = self.load_crawl_state(keywords)
        
        print(f"\n🚀 (阶段1) 开始/继续抓取职位列表...")
        
        for kw_idx in range(start_kw_idx, len(keywords)):
            keyword = keywords[kw_idx]
            page = start_page if kw_idx == start_kw_idx else 1
            
            print(f"\n{'='*40}")
            print(f"关键词 ({kw_idx+1}/{len(keywords)}): {keyword}")
            print(f"{'='*40}")
            
            success = self.fetch_data(query=keyword, start_page=page, max_pages=max_pages_per_kw)
            
            if not success:
                print("\n⚠️ 触发风控，已保存当前进度。")
                return
            
            if kw_idx < len(keywords) - 1:
                time.sleep(random.uniform(3.0, 5.0))
        
        print("\n👍 (阶段1) 职位列表抓取完毕。")
        
        if self.data_list:
            self.fetch_all_details_resumable(start_index=start_detail_index)
        
        self.save_final_excel()
        
        if os.path.exists(self.state_file):
            os.remove(self.state_file)
            print(f"   🗑️  已删除状态文件 {self.state_file}")

if __name__ == "__main__":
    spider = BossSpiderAPI()
    
    keywords = ["银行", "保险", "数据分析", "基金经理", "产品", "运营", "证券研究所", "投行", "金融"]
    # 保持顺序去重
    
    # 每个关键词抓取
    
    spider.run(keywords=keywords, max_pages_per_kw=30)


🔄 正在从 job_data_IN_PROGRESS.xlsx 加载进度...
✅ 加载成功，已恢复 1755 条记录。
   其中 0 条已有描述，1755 条待抓取。
📍 恢复爬取进度：关键词 [银行] 第 1 页

🚀 (阶段1) 开始/继续抓取职位列表...

关键词 (1/9): 银行

🔍 抓取关键词 [银行] 列表第 1 页...
⚠️ 触发风控 (code: 37)！
📍 当前进度：关键词 [银行] 第 1 页

⚠️ 触发风控，已保存当前进度。
